## Metaprogramming with Metaclasses in Python

At first word Metaprogramming seems very funky and alien thing but if you have ever worked with decorators or metaclasses, your were doing metaprogramming there. In nutshell we can say metaprogramming is the code which manipulates code.

In this article we are going to discuss about Metaclasses, why and when we should use them and what are the alternatives. This is fairly advance Python topic and following prerequisite is expected –

OOP concept in Python
Decorators in Python

### Metaclasses

In Python everything have some type associated with it. For example if we have a variable having integer value then it’s type is int. You can get type of anything using type() function.

In [1]:
num = 23
print("Type of num is:", type(num)) 
  
lst = [1, 2, 4] 
print("Type of lst is:", type(lst)) 
  
name = "Atul"
print("Type of name is:", type(name)) 

Type of num is: <class 'int'>
Type of lst is: <class 'list'>
Type of name is: <class 'str'>


Every type in Python is defined by Class. So in above example, unlike C or Java where int, char, float are primary data types, in Python they are object of int class or str class. So we can make a new type by creating a class of that type. For example we can create a new type Student by creating Student class.

In [2]:
class Student:
    pass

std_obj = Student()
print("type of std_obj is: ", type(std_obj))

type of std_obj is:  <class '__main__.Student'>


A Class is also an object, and just like any other object it’s a instance of something called Metaclass. A special class type creates these Class object. The type class is default metaclass which is responsible for making classes. For example in above example if we try to find out the type of Student class, it comes out to be a type.

In [3]:
print("type of Student is: ", type(Student))

type of Student is:  <class 'type'>


Because Classes are also an object, they can be modified in same way. We can add or subtract fields or methods in class in same way we did with other objects. For example –

In [5]:
# Defined class without any 
# class methods and variables 
class Test:
    pass

# Defining method variables 
Test.x = 45

# Defining class methods
Test.foo = lambda self: print("hello")

# Creating object
my_obj = Test()
print(my_obj.x)
my_obj.foo()

45
hello


This whole meta thing can be summarized as – Metaclass create Classes and Classes creates objects

Metaclass is responsible for generation of classes, so we can write our own custom metaclasses to modify the way classes are generated by performing extra actions or injecting code. Usually we do not need custom metaclasses but sometime it’s necessary.
There are problems for which metaclass and non-metaclass based solutions are available (often simpler) but in some cases only metaclass can solve the problem. We will discuss such problem in this article.

To create our custom metaclass, our custom metaclass have to inherit type metaclass and usually override –
1. \__new__(): It’s a method which is called before __init__(). It creates the object and return it. We can overide this method to control how the objects are created.
2. \__init__(): This method just initialize the created object passed as parameter

We can create classes using type() function directly. It can be called in following ways –

1. When called with only one argument, it returns the type. We have seen it before in above examples.
1. When called with three parameters, it creates a class. Following arguments are passed to it –
    1. Class name
    2. Tuple having base classes inherited by class
    3. Class Dictionary: It serves as local namespace for the class, populated with class methods and variables

In [19]:
def test_method(self):
    print("this is Test class Method")
    
class Base:
    def myfun(self):
        print("this is inherited method")

# Creating Test class dynamically using 
# type() method directly 
Test = type('Test',(Base, ), dict(x="Rahul", my_method=test_method))

print('Type of test class', type(Test))

my_obj = Test()

print('Type of my_obj', type(my_obj))

my_obj.myfun()
my_obj.my_method()

print(my_obj.x)

print(issubclass(Test, Base))

Type of test class <class 'type'>
Type of my_obj <class '__main__.Test'>
this is inherited method
this is Test class Method
Rahul
True


Now let’s create a metaclass without using type() directly. In the following example we will be creating a metaclass MultiBases which will check if class being created have inherited from more than one base classes. If so, it will raise an error.

In [13]:
class MultiBases(type):
    def __new__(cls, clsname, bases, clsdict):
        # if no of base classes is greator than 1 
        # raise error 
        if len(bases) > 1:
            raise TypeError("Inherited multiple base classes !!!")
            
        # else execute __new__ method of super class, ie. 
        # call __init__ of type class 
        
        return super().__new__(cls, clsname, bases, clsdict)
    
# metaclass can be specified by 'metaclass' keyword argument 
# now MultiBase class is used for creating classes 
# this will be propagated to all subclasses of Base 

class Base(metaclass=MultiBases):
    pass


# no error is raised
class A(Base):
    pass

# no error is raised
class B(Base):
    pass

# this will raise an error
# class C(A, B):
#     pass

<class '__main__.MultiBases'>


### Solving problem with metaclass

There are some problems which can be solved by decorators (easily) as well as by metaclasses. But there are few problems whose result can only be achived by metaclasses. For example consider a very simple problem of code repetition.


We want to debug class methods, what we want is that whenever class method executes, it should print it’s fully qualified name before executing it’s body.

In [37]:
def decoratemet(func):
    def wrapper():
        return func.__qualname__ + ' '+ str(func())
    
    return wrapper
        
@decoratemet
def hello():
    return "Rahul"
    
hello()



'hello Rahul'

In [38]:
from functools import wraps 

def debug(func): 
	'''decorator for debugging passed function'''
	
	@wraps(func) 
	def wrapper(*args, **kwargs): 
		print("Full name of this method:", func.__qualname__) 
		return func(*args, **kwargs) 
	return wrapper 

def debugmethods(cls): 
	'''class decorator make use of debug decorator 
	to debug class methods '''
	
	# check in class dictionary for any callable(method) 
	# if exist, replace it with debugged version 
	for key, val in vars(cls).items(): 
		if callable(val): 
			setattr(cls, key, debug(val)) 
	return cls

# sample class 
@debugmethods
class Calc: 
	def add(self, x, y): 
		return x+y 
	def mul(self, x, y): 
		return x*y 
	def div(self, x, y): 
		return x/y 
	
mycal = Calc() 
print(mycal.add(2, 3)) 
print(mycal.mul(5, 2)) 

Full name of this method: Calc.add
5
Full name of this method: Calc.mul
10


In [11]:
def debug(func):
    def wrapper(*args, **kwargs):
        print("full name of this method", func.__qualname__)
        return func(*args, **kwargs)
    return wrapper

def debugmethods(cls):
#     print(vars(cls).items())
    for k, v in vars(cls).items():
        if callable(v):
            setattr(cls, k, debug(v))

    return cls

@debugmethods
class Math:
    def add(self, x, y):
        return x + y

m = Math()
m.add(3,4)


full name of this method Math.add


7

This solution works fine but there is one problem, what if we want to apply this method decorator to all subclasses which inherit this Calc class. In that case we have to separately apply method decorator to every subclass just like we did with Calc class.

The problem is if we have many such subclasses, then in that case we won’t like adding decorator to each one separately. If we know beforehand that every subclass must have this debug property, then we should look up to metaclass based solution.

Have a look at this metaclass based solution, the idea is that classes will be created normally and then immediately wrapped up by debug method decorator –

In [15]:
def debug(func):
    def wrapper(*args, **kwargs):
        print("full name of this method", func.__qualname__)
        return func(*args, **kwargs)
    return wrapper

def debugmethods(cls):
    for k, v in vars(cls).items():
        if callable(v):
            setattr(cls, k, debug(v))

    return cls

class debugMeta(type):
    '''meta class which feed created class object 
   to debugmethod to get debug functionality 
   enabled objects'''
    def __new__(cls, clsname, bases, clsdict):
        obj = super().__new__(cls, clsname, bases, clsdict)
        obj = debugmethods(obj)
        return obj

# base class with metaclass 'debugMeta' 
# now all the subclass of this  
# will have debugging applied 
class Base(metaclass=debugMeta):
    pass

class Calc(Base):
    def add(self, x, y):
        return x + y
    
    def mul(self, x, y):
        return x*y
    
myCalc = Calc()
print(myCalc.add(7,8))
print(myCalc.mul(9,5))
        

full name of this method Calc.add
15
full name of this method Calc.mul
45


### When to use Metaclasses

Most of the time we are not using metaclasses, they are like black magic and usually for something complicated, but few cases where we use metaclasses are –

* As we have seen in above example, metaclasses propogate down the inheritance hierarchies. It will affect all the subclasses as well. If we have such situation, then we should use metaclasses.
* If we want to change class automatically, when it is created
* If you are API developer, you might use metaclasses

As quoted by Tim Peters,  
Metaclasses are deeper magic that 99% of users should never worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why).